📘 1️⃣ Project Introduction

# 🧠 AI-Powered Mental Health Support & Crisis-Alert Chatbot

This project is a compassionate, AI-assisted mental-health support tool that:

• Provides empathetic, conversational responses  
• Detects crisis or self-harm signals and shows emergency helplines  
• Guides users through coping exercises  
  – Breathing  
  – Grounding (5-4-3-2-1)  
  – Stress relief  
  – Self-care prompts  
• Tracks mood throughout the conversation  
• Includes a personalized intake form  
• Supports conversation memory (per-session)  
• Runs as a Flask web app with modern UI + animations  

The notebook installs dependencies, creates the backend and UI,
and launches the chat interface via ngrok.


📘 2️⃣ Install Dependencies

This step installs:

• Flask → Backend web framework  
• pyngrok → Public URL for your app  
• Transformers → AI conversational model  
• Torch → Model inference  
• SentencePiece → Tokenizer support  
• Accelerate → Faster inference  

It also creates required folders:
templates/ and static/


In [ ]:
# ===============================
# 1️⃣ Install Dependencies
# ===============================
!pip install -q flask pyngrok transformers torch sentencepiece accelerate
!mkdir -p templates static

📘 3️⃣ Flask Backend Overview

The backend handles:

1. Intake Form  
   • Name  
   • Mood  
   • Primary concern  
   • Age  
   • Therapy history  

2. Conversation Engine  
   • Custom empathetic response generator  
   • Rule-based detection of:
     – Anxiety  
     – Depression  
     – Loneliness  
     – Low self-esteem  
     – Stress and burnout  
     – Relationship issues  
     – Sleep issues  
     – Trauma / past experiences  

3. Crisis Detection  
   Scans messages for:
   – “suicide”, “want to die”, “hurt myself”, etc.  
   Immediately shows **emergency helpline response**.

4. Conversation Memory  
   Stores per-session:
   • User messages  
   • Bot responses  
   • Mood changes  

5. Mood Tracking API  
   Allows user to update how they feel anytime.

6. Session Management  
   Clears history on session end.


📘 4️⃣ AI Model Overview

The app attempts to load:

✔ **microsoft/DialoGPT-medium**  
   – Specialized for conversations  
   – More empathetic natural replies  

If it fails:

✔ Fallback: **facebook/blenderbot-400M-distill**

Both models run automatically on GPU if available.


📘 5️⃣ Empathy & Mental-Health Logic

The chatbot is NOT just a normal LLM.

It uses:
• Conversation memory  
• Personalized responses using user name, mood, concern  
• Rule-based emotional support modules:
  – Anxiety  
  – Depression  
  – Stress  
  – Loneliness  
  – Relationships  
  – Sleep problems  
  – Self-esteem / confidence  

Special modules:
• Guided breathing  
• Grounding technique  
• Sleep hygiene plan  
• Quick stress-relief toolkit  

Crisis detection always overrides everything
and returns emergency support resources.


In [ ]:
# ===============================
# 2️⃣ Create Enhanced Flask App
# ===============================
%%writefile app.py
from flask import Flask, render_template, request, jsonify, session
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re
from datetime import datetime
from collections import defaultdict

app = Flask(__name__)
app.secret_key = "mental_health_support_enhanced_2024_secure_key"
app.config['SESSION_TYPE'] = 'filesystem'

# Initialize better AI model for conversational responses
print("🔄 Loading Enhanced Mental Health Support AI Model (DialoGPT)...")

try:
    # Using microsoft/DialoGPT-medium for better conversational AI
    model_name = "microsoft/DialoGPT-medium"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None
    )
    print("✅ DialoGPT Model loaded successfully!")
except Exception as e:
    print(f"⚠️ DialoGPT loading failed, using fallback: {e}")
    try:
        # Fallback to BlenderBot
        model_name = "facebook/blenderbot-400M-distill"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        print("✅ BlenderBot Model loaded successfully!")
    except Exception as e2:
        print(f"⚠️ All models failed: {e2}")
        model = None
        tokenizer = None

# Store conversation history with memory
conversation_history = defaultdict(list)
mood_history = defaultdict(list)

def get_conversation_context(session_id, limit=5):
    """Get recent conversation context for memory"""
    if session_id not in conversation_history:
        return ""

    recent = conversation_history[session_id][-limit:]
    context = ""
    for msg in recent:
        context += f"User: {msg['user']}\nBot: {msg['bot'][:100]}...\n"
    return context

def get_empathetic_response(user_message, user_data, session_id):
    """Generate empathetic mental health support response with conversation memory"""

    # Extract user information
    name = user_data.get('name', 'friend')
    concern = user_data.get('concern', 'general wellbeing')
    mood = user_data.get('mood', 'neutral')

    # Get conversation context for memory
    context = get_conversation_context(session_id, limit=3)

    # Rule-based responses for common mental health queries
    user_message_lower = user_message.lower()

    # Check for follow-up references
    is_followup = any(word in user_message_lower for word in ['yes', 'yeah', 'okay', 'tell me more', 'continue', 'go on', 'please', 'help me with'])

    # Crisis detection
    crisis_keywords = ['suicide', 'kill myself', 'end my life', 'want to die', 'hurt myself', 'self harm']
    if any(keyword in user_message_lower for keyword in crisis_keywords):
        return """🆘 **IMMEDIATE CRISIS SUPPORT NEEDED**

Your life matters, and there are people who want to help you RIGHT NOW.

**🚨 CALL IMMEDIATELY:**
• **988** - Suicide & Crisis Lifeline (US) - 24/7
• **911** - If in immediate danger
• **Crisis Text Line**: Text **HOME** to **741741**
• **International**: https://findahelpline.com

**You are not alone.** Professional crisis counselors are trained to help in these moments. Please reach out to them immediately.

I care about you and I'm here, but trained crisis professionals can provide the immediate, specialized support you need right now. 💙"""

    # Breathing exercise request
    if any(word in user_message_lower for word in ['breathing', 'breathe', 'calm down', 'panic attack', 'anxious right now']):
        return f"""**🌬️ Let's Do a Calming Breathing Exercise Together, {name}**

I'll guide you through the **4-7-8 Technique** - a powerful method to calm anxiety:

**Instructions:**
1️⃣ **Breathe IN** through your nose for **4 seconds**
2️⃣ **HOLD** your breath for **7 seconds**
3️⃣ **Breathe OUT** through your mouth for **8 seconds**

**Let's start:**
• Get comfortable, close your eyes if you can
• Place one hand on your belly
• Ready? Begin...

**Breathe in... 1... 2... 3... 4...**
**Hold... 1... 2... 3... 4... 5... 6... 7...**
**Breathe out... 1... 2... 3... 4... 5... 6... 7... 8...**

**Repeat this 4 times.**

How do you feel now? Would you like to try another calming technique?"""

    # Grounding technique request
    if any(word in user_message_lower for word in ['grounding', '5-4-3-2-1', 'overwhelmed', 'disconnected', 'panic']):
        return f"""**🧘 Let's Try the 5-4-3-2-1 Grounding Technique, {name}**

This helps bring you back to the present moment when feeling overwhelmed:

**Look around you and identify:**

**5️⃣ FIVE things you can SEE**
(Example: a lamp, a book, your phone, a plant, the ceiling)

**4️⃣ FOUR things you can TOUCH**
(Example: your chair, your clothes, the floor, your hair)

**3️⃣ THREE things you can HEAR**
(Example: birds outside, traffic, your breathing)

**2️⃣ TWO things you can SMELL**
(Example: fresh air, coffee, perfume)

**1️⃣ ONE thing you can TASTE**
(Example: mint, water, your lunch)

Take your time with each sense. How do you feel now?"""

    # Depression support with memory
    if any(word in user_message_lower for word in ['depressed', 'depression', 'hopeless', 'empty', 'numb', 'no energy']):
        memory_note = ""
        if context and 'depression' in context.lower():
            memory_note = "\n\nI remember you mentioned feeling this way earlier. How has it been since we last talked about this?"

        return f"""**💙 I hear you, {name}, and I want you to know that what you're feeling is valid.**

Depression can make everything feel overwhelming and exhausting. It's not your fault.{memory_note}

**🌟 What might help right now:**
• **Talk to a professional** - They can provide proper diagnosis and treatment options
• **Small self-care acts**: Get 10 minutes of sunlight, eat one healthy meal, drink water
• **Reach out** to one trusted friend or family member
• **Therapy options**: CBT, interpersonal therapy, or medication if recommended by a doctor
• **Support groups**: Connect with others who understand

**⚡ Quick relief activities:**
• Take a 5-minute walk outside
• Listen to your favorite song
• Do ONE small task (make your bed, wash one dish)
• Write down ONE thing you're grateful for

**Remember:** Depression is a medical condition, not a weakness. With proper support and treatment, people DO get better. You deserve that support.

What specific aspect would you like to explore more? Or would you like to try one of the quick relief activities together?"""

    # Anxiety support with interactive options
    if any(word in user_message_lower for word in ['anxious', 'anxiety', 'worried', 'nervous', 'cant stop thinking']):
        memory_note = ""
        if context and 'anxiety' in context.lower():
            memory_note = "\n\nI see you're still experiencing anxiety. Let's work through this together."

        return f"""**🌊 Thank you for sharing, {name}. Anxiety can feel incredibly overwhelming, but there ARE ways to manage it.**{memory_note}

**🆘 Need immediate relief? Try these:**
• **[Breathing Exercise]** - 4-7-8 technique (click to try)
• **[Grounding Exercise]** - 5-4-3-2-1 method (click to try)
• **Progressive Muscle Relaxation** - Tense and release each muscle group

**💪 Longer-term strategies:**
• **Regular exercise** (even 10-minute walks release endorphins)
• **Limit caffeine & alcohol** (they can worsen anxiety)
• **Sleep routine**: Same bedtime, cool room, no screens 1hr before bed
• **Mindfulness/meditation**: Try Headspace, Calm, or Insight Timer apps
• **CBT therapy**: Very effective for anxiety - consider finding a therapist

**📝 Anxiety journal technique:**
Write down:
1. What am I worried about?
2. Is this worry realistic or catastrophizing?
3. What's the worst that could happen? Best? Most likely?
4. What can I control? What can't I?

Would you like to:
• Try a breathing exercise together? (say "breathing")
• Talk about what's triggering your anxiety?
• Learn more coping strategies?"""

    # Stress management with personalization
    if any(word in user_message_lower for word in ['stressed', 'stress', 'pressure', 'burnout', 'overwhelmed with work']):
        memory_note = ""
        if context and 'stress' in context.lower():
            memory_note = f"\n\nYou mentioned stress before. Has anything changed, {name}?"

        return f"""**🌈 I understand you're dealing with a lot of stress, {name}.** It's important that you're acknowledging this.{memory_note}

**⚡ QUICK STRESS RELIEF (Do RIGHT NOW):**
• Take **5 deep breaths** (in for 4, hold for 4, out for 6)
• **Stand up and stretch** for 30 seconds
• **Drink a glass of water**
• **Step away** from the stressor for 5 minutes

**🛠️ COPING TECHNIQUE TOOLKIT:**
• **[Box Breathing]** - Try this guided exercise
• **[Body Scan]** - Release tension in your muscles
• **[5-Minute Walk]** - Get outside if possible

**📋 STRESS MANAGEMENT STRATEGIES:**
• **Break tasks down**: Large tasks → smaller, manageable steps
• **Set boundaries**: It's OKAY to say no
• **Schedule breaks**: Every 90 minutes, take 10-15 min break
• **Connect**: Talk to supportive people
• **Self-care time**: Block 30 min daily for activities you enjoy
• **Sleep**: Prioritize 7-9 hours (stress + lack of sleep = worse stress)

**🎯 Let's tackle this together:**
What's the **main source** of stress for you right now? Let's see if we can break it down into manageable pieces.

Or would you like to try a **stress-relief technique** first?"""

    # Loneliness/isolation with actionable steps
    if any(word in user_message_lower for word in ['lonely', 'alone', 'isolated', 'no friends', 'nobody cares']):
        return f"""**💕 Feeling lonely can be really painful, {name}.** I want you to know that you're not alone in feeling this way, and there ARE paths forward.

**🌟 WAYS TO BUILD CONNECTION:**

**🎯 Start Small (This Week):**
• Message **one** old acquaintance: "Hey, how have you been?"
• Join **one** online community for your interests (Reddit, Discord, Facebook groups)
• Say "hi" to a neighbor or coworker
• Comment on social media posts from people you know

**🚀 Next Steps (This Month):**
• **Join local groups**: Meetup.com, community centers, hobby clubs
• **Take a class**: Art, cooking, fitness, language learning
• **Volunteer**: Animal shelters, food banks, community gardens
• **Sports/fitness groups**: Running clubs, yoga classes, rec sports

**💻 Online Communities:**
• Discord servers for your hobbies
• Reddit communities (r/MakeNewFriendsHere, r/Needafriend)
• Online gaming communities
• Virtual book clubs

**🐾 Other Options:**
• **Pet companionship**: Dogs, cats, even fish can help reduce loneliness
• **Therapy**: A therapist can help work through these feelings
• **Support groups**: In-person or online for people experiencing similar feelings

**💡 IMPORTANT TO REMEMBER:**
• Building connections takes time - be patient with yourself
• Start with ONE small action this week
• Quality > Quantity - even one good friend makes a huge difference
• Rejection is normal - keep trying, you WILL find your people

What activities or interests do you have, {name}? Let's think about where you might find people with similar interests.

Or would you like help crafting a message to reach out to someone?"""

    # Sleep issues with structured program
    if any(word in user_message_lower for word in ['sleep', 'insomnia', 'tired', 'exhausted', 'cant sleep', 'awake at night']):
        return f"""**😴 Sleep difficulties can really impact mental health, {name}.** Let's create a better sleep plan for you.

**🌙 SLEEP HYGIENE PROGRAM:**

**⏰ SCHEDULE (Most Important!):**
• **Same bedtime & wake time** EVERY day (yes, weekends too)
• Your body needs consistency to regulate sleep hormones
• Aim for 7-9 hours

**🛏️ BEDTIME ROUTINE (1 Hour Before Sleep):**
• **9:00 PM**: Dim lights, no more screens
• **9:15 PM**: Relaxing activity (reading, gentle stretching, meditation)
• **9:30 PM**: Warm shower or bath
• **9:45 PM**: Light snack if hungry (avoid heavy meals)
• **10:00 PM**: Bed (adjust times to your schedule)

**🚫 WHAT TO AVOID:**
• **No caffeine** after 2 PM (coffee, tea, soda, chocolate)
• **No alcohol** close to bedtime (disrupts sleep cycles)
• **No heavy meals** 3 hours before bed
• **No intense exercise** 4 hours before bed (light stretching is OK)
• **No screens** 1 hour before bed (blue light blocks melatonin)

**✅ BEDROOM OPTIMIZATION:**
• **Temperature**: 60-67°F (cool is better)
• **Darkness**: Blackout curtains or eye mask
• **Quiet**: White noise machine or earplugs
• **Comfort**: Good mattress, pillow, breathable sheets

**🧘 RELAXATION TECHNIQUES:**
• **Progressive muscle relaxation**: Tense & release each muscle group
• **4-7-8 breathing**: In for 4, hold 7, out for 8
• **Body scan meditation**: Focus on relaxing each body part
• **Visualization**: Imagine a peaceful, calming place

**⚠️ WHEN TO SEE A DOCTOR:**
If sleep problems persist for more than 3 weeks, talk to your doctor. Insomnia can be a symptom of other conditions (anxiety, depression, sleep apnea).

What's your **current bedtime routine** like, {name}? Let's identify what we can improve together.

Would you like me to create a **personalized sleep schedule** for you?"""

    # Relationship issues with empathy
    if any(word in user_message_lower for word in ['relationship', 'breakup', 'divorce', 'family', 'partner', 'spouse', 'boyfriend', 'girlfriend']):
        return f"""**💔 Relationship challenges can be emotionally draining, {name}.** I'm here to listen without judgment.

**🗣️ HEALTHY RELATIONSHIP PRACTICES:**

**Communication:**
• **"I feel..."** statements instead of "You always..."
• **Active listening**: Really hear them without planning your response
• **No blame game**: Focus on solving problems together
• **Regular check-ins**: "How are we doing?" conversations

**Boundaries:**
• **Know your limits**: What's acceptable and what's not
• **Communicate boundaries** clearly and kindly
• **Respect their boundaries** too
• **It's OK to say no**

**Conflict Resolution:**
• **Take breaks** if things get heated (20-min cooldown)
• **No name-calling** or personal attacks
• **Focus on the issue**, not attacking character
• **Compromise**: Both people need to feel heard

**Self-Care in Relationships:**
• **Maintain your identity** outside the relationship
• **Keep friendships** and hobbies
• **Set aside "me time"**
• **Don't lose yourself** trying to please them

**🚨 RED FLAGS (When to Leave):**
• **Physical, emotional, or verbal abuse**
• **Controlling behavior** (isolating you from friends/family)
• **Constant disrespect or criticism**
• **Refusal to communicate or work on issues**
• **Repeated cheating or betrayal**

**💚 RESOURCES:**
• **Couples/Family Therapy**: Can help tremendously
• **National Domestic Violence Hotline**: 1-800-799-7233 (if there's abuse)
• **Text START to 88788** (for domestic violence support)

Would you like to talk more about what's happening, {name}?

What specific aspect of your relationship is most challenging right now?"""

    # Self-esteem/confidence building
    if any(word in user_message_lower for word in ['confidence', 'self-esteem', 'worthless', 'not good enough', 'failure', 'hate myself']):
        return f"""**💪 Your feelings matter, {name}, and I want to help you see your worth.** You are enough, exactly as you are.

**🌟 BUILDING SELF-ESTEEM:**

**1️⃣ CHALLENGE NEGATIVE SELF-TALK:**
• **Negative thought**: "I'm a failure"
• **Challenge it**: "Is this really true? Would I say this to a friend?"
• **Reframe**: "I made a mistake, but that doesn't define me. I'm learning and growing."

**2️⃣ EVIDENCE JOURNAL:**
Write down **daily**:
• 3 things you did well today (even tiny things count!)
• 1 compliment you received
• 1 challenge you faced (you survived it!)

**3️⃣ SELF-COMPASSION PRACTICE:**
• **Talk to yourself** like you'd talk to a best friend
• **Acknowledge pain**: "This is hard, and that's okay"
• **Common humanity**: "Everyone struggles sometimes"
• **Be kind to yourself**: You deserve gentleness

**4️⃣ ACCOMPLISHMENT LIST:**
List **everything** you've accomplished (education, jobs, skills, relationships, times you helped others, challenges overcome)
• You've done more than you give yourself credit for!

**5️⃣ SMALL ACHIEVABLE GOALS:**
• Set **ONE small goal** per day
• **Celebrate** when you achieve it
• Build momentum: Success → Confidence → More success

**6️⃣ LIMIT COMPARISONS:**
• **Social media detox**: Take a break or unfollow accounts that make you feel bad
• **Remember**: People only show highlights, not struggles
• **Your journey is unique** - don't compare your Chapter 3 to someone's Chapter 20

**🎯 LET'S START NOW:**
Tell me **ONE thing** you accomplished recently, no matter how small, {name}. It could be:
• Got out of bed
• Made a meal
• Responded to a message
• Went to work/school
• Anything at all!

**💡 THERAPY RECOMMENDATION:**
Consider therapy (especially CBT) to address deep-rooted beliefs about yourself. It can be incredibly helpful.

What would you like to work on first?"""

    # Positive responses with encouragement
    if any(word in user_message_lower for word in ['better', 'feeling good', 'happy', 'great', 'thanks', 'helped', 'thank you']):
        return f"""**🎉 I'm SO glad to hear that, {name}!** 😊

This is wonderful progress! Celebrating these moments is so important.

**🌟 KEEP THE MOMENTUM GOING:**
• **Acknowledge** your hard work - YOU did this!
• **Practice gratitude** - write down 3 good things each day
• **Continue strategies** that are working for you
• **Be patient** with yourself on harder days (they'll come, and that's OK)

**💡 REMEMBER:**
• Progress isn't linear - ups and downs are normal
• Every small step counts
• You're building resilience and coping skills
• Keep reaching out when you need support

Is there anything else you'd like to talk about? Or would you like to explore more strategies to maintain this positive momentum?

I'm here whenever you need me! 💙"""

    # General supportive response with conversation memory
    memory_context = ""
    if context:
        memory_context = "\n\nI remember we've been talking about your concerns. How have things been going since then?"

    return f"""**Thank you for sharing that with me, {name}.** I'm here to listen and support you through whatever you're experiencing.{memory_context}

**💙 I WANT YOU TO KNOW:**
• **Your feelings are valid** - what you're experiencing matters
• **It's okay to not be okay** - struggling doesn't make you weak
• **Seeking help is strength** - reaching out takes courage
• **Recovery and healing are possible** - with support, things can get better

Could you tell me more about what you're going through? What's been most challenging for you regarding your **{concern}**?

**🆘 PROFESSIONAL RESOURCES:**
If you need professional support:
• **SAMHSA National Helpline**: **1-800-662-4357** (24/7, free, confidential)
• **Psychology Today**: Find a therapist near you
• **BetterHelp/Talkspace**: Online therapy options
• **Crisis Text Line**: Text HOME to **741741**

**🎯 OR TRY A COPING TECHNIQUE:**
• Say **"breathing"** for a guided calming exercise
• Say **"grounding"** for the 5-4-3-2-1 technique
• Say **"stress relief"** for quick tension release methods

I'm here for you, {name}. What would be most helpful for you right now?"""

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/submit_intake", methods=["POST"])
def submit_intake():
    """Store user intake information and redirect to chat"""
    try:
        data = request.json

        session_id = request.cookies.get('session', str(datetime.now().timestamp()))

        # Store in session
        session['user_data'] = {
            'name': data.get('name'),
            'age': data.get('age'),
            'concern': data.get('concern'),
            'mood': data.get('mood'),
            'previous_therapy': data.get('previous_therapy'),
            'timestamp': datetime.now().isoformat(),
            'session_id': session_id
        }

        # Initialize conversation history
        if session_id not in conversation_history:
            conversation_history[session_id] = []

        # Initialize mood history
        if session_id not in mood_history:
            mood_history[session_id] = []

        # Store initial mood
        mood_history[session_id].append({
            'mood': data.get('mood'),
            'timestamp': datetime.now().isoformat()
        })

        return jsonify({"status": "success", "redirect": "/chat"})

    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route("/chat")
def chat():
    """Chat interface page"""
    if 'user_data' not in session:
        return render_template("index.html")
    return render_template("chat.html", user_data=session['user_data'])

@app.route("/send_message", methods=["POST"])
def send_message():
    """Handle chat messages"""
    try:
        data = request.json
        user_message = data.get('message', '').strip()

        if not user_message:
            return jsonify({"error": "Empty message"}), 400

        # Get user data from session
        user_data = session.get('user_data', {})
        session_id = user_data.get('session_id', str(datetime.now().timestamp()))

        # Generate response with conversation memory
        bot_response = get_empathetic_response(user_message, user_data, session_id)

        # Store in conversation history
        if session_id not in conversation_history:
            conversation_history[session_id] = []

        conversation_history[session_id].append({
            'user': user_message,
            'bot': bot_response,
            'timestamp': datetime.now().isoformat()
        })

        return jsonify({
            "response": bot_response,
            "status": "success"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/update_mood", methods=["POST"])
def update_mood():
    """Track mood changes throughout conversation"""
    try:
        data = request.json
        mood = data.get('mood')

        user_data = session.get('user_data', {})
        session_id = user_data.get('session_id', str(datetime.now().timestamp()))

        if session_id not in mood_history:
            mood_history[session_id] = []

        mood_history[session_id].append({
            'mood': mood,
            'timestamp': datetime.now().isoformat()
        })

        return jsonify({"status": "success"})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/get_mood_history", methods=["GET"])
def get_mood_history():
    """Get mood tracking data"""
    try:
        user_data = session.get('user_data', {})
        session_id = user_data.get('session_id', str(datetime.now().timestamp()))

        history = mood_history.get(session_id, [])

        return jsonify({
            "mood_history": history,
            "status": "success"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/end_session", methods=["POST"])
def end_session():
    """End chat session"""
    session.clear()
    return jsonify({"status": "success"})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)

📘 6️⃣ Intake Form Interface (index.html)

The intake form includes:

✔ Name  
✔ Age (optional)  
✔ Main concern  
✔ Mood selector with emojis  
✔ Therapy history  
✔ Crisis warning banner  

This form initializes:
• user_data in session  
• starting mood  
• conversation history  


In [ ]:
# ===============================
# 3️⃣ HTML - Intake Form
# ===============================
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🧠 Mental Health Support Chat</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Inter', 'Segoe UI', system-ui, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            display: flex;
            align-items: center;
            justify-content: center;
        }

        .container {
            max-width: 600px;
            width: 100%;
            background: white;
            padding: 50px;
            border-radius: 25px;
            box-shadow: 0 20px 60px rgba(0, 0, 0, 0.3);
            animation: fadeIn 0.6s ease;
        }

        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(30px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .header {
            text-align: center;
            margin-bottom: 40px;
        }

        .logo {
            font-size: 70px;
            margin-bottom: 15px;
            animation: pulse 2s ease infinite;
        }

        @keyframes pulse {
            0%, 100% { transform: scale(1); }
            50% { transform: scale(1.05); }
        }

        h1 {
            font-size: 32px;
            color: #667eea;
            margin-bottom: 10px;
            font-weight: 700;
        }

        .subtitle {
            font-size: 16px;
            color: #718096;
            line-height: 1.6;
        }

        .form-group {
            margin-bottom: 25px;
        }

        label {
            display: block;
            font-weight: 600;
            color: #2d3748;
            margin-bottom: 8px;
            font-size: 15px;
        }

        input, select, textarea {
            width: 100%;
            padding: 14px;
            border: 2px solid #e2e8f0;
            border-radius: 10px;
            font-size: 15px;
            transition: all 0.3s ease;
            font-family: inherit;
        }

        input:focus, select:focus, textarea:focus {
            outline: none;
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }

        textarea {
            resize: vertical;
            min-height: 100px;
        }

        .mood-options {
            display: grid;
            grid-template-columns: repeat(5, 1fr);
            gap: 10px;
            margin-top: 10px;
        }

        .mood-btn {
            padding: 15px 5px;
            border: 2px solid #e2e8f0;
            border-radius: 10px;
            background: white;
            cursor: pointer;
            transition: all 0.3s ease;
            font-size: 24px;
            text-align: center;
        }

        .mood-btn:hover {
            border-color: #667eea;
            transform: translateY(-2px);
        }

        .mood-btn.selected {
            border-color: #667eea;
            background: #f7fafc;
        }

        .submit-btn {
            width: 100%;
            padding: 18px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 12px;
            font-size: 18px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s ease;
            margin-top: 10px;
        }

        .submit-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 25px rgba(102, 126, 234, 0.4);
        }

        .privacy-note {
            margin-top: 25px;
            padding: 15px;
            background: #f7fafc;
            border-radius: 10px;
            font-size: 13px;
            color: #718096;
            line-height: 1.6;
        }

        .crisis-banner {
            background: #fed7d7;
            color: #c53030;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 25px;
            font-size: 14px;
            line-height: 1.6;
            font-weight: 500;
        }

        @media (max-width: 768px) {
            .container { padding: 30px 25px; }
            h1 { font-size: 26px; }
            .mood-options { grid-template-columns: repeat(5, 1fr); }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <div class="logo">🧠</div>
            <h1>Mental Health Support</h1>
            <p class="subtitle">Welcome. This is a safe space to share your thoughts and feelings. Let's start with a few questions to help me understand how to support you better.</p>
        </div>

        <div class="crisis-banner">
            <strong>🆘 In Crisis?</strong> If you're in immediate danger, please call 988 (Suicide & Crisis Lifeline) or 911. For non-emergency support: SAMHSA 1-800-662-4357
        </div>

        <form id="intakeForm">
            <div class="form-group">
                <label for="name">What should I call you? *</label>
                <input type="text" id="name" name="name" placeholder="Your name or nickname" required>
            </div>

            <div class="form-group">
                <label for="age">Age (optional)</label>
                <input type="number" id="age" name="age" placeholder="Your age" min="13" max="120">
            </div>

            <div class="form-group">
                <label for="concern">What brings you here today? *</label>
                <select id="concern" name="concern" required>
                    <option value="">Select your primary concern...</option>
                    <option value="anxiety">Anxiety / Worry</option>
                    <option value="depression">Depression / Low Mood</option>
                    <option value="stress">Stress / Overwhelm</option>
                    <option value="relationships">Relationship Issues</option>
                    <option value="loneliness">Loneliness / Isolation</option>
                    <option value="self-esteem">Self-Esteem / Confidence</option>
                    <option value="trauma">Trauma / Past Experiences</option>
                    <option value="sleep">Sleep Problems</option>
                    <option value="work">Work / School Stress</option>
                    <option value="grief">Grief / Loss</option>
                    <option value="other">Other / General Support</option>
                </select>
            </div>

            <div class="form-group">
                <label>How are you feeling right now? *</label>
                <div class="mood-options">
                    <div class="mood-btn" data-mood="very-sad" title="Very Sad">😢</div>
                    <div class="mood-btn" data-mood="sad" title="Sad">😔</div>
                    <div class="mood-btn" data-mood="neutral" title="Neutral">😐</div>
                    <div class="mood-btn" data-mood="good" title="Good">🙂</div>
                    <div class="mood-btn" data-mood="great" title="Great">😊</div>
                </div>
                <input type="hidden" id="mood" name="mood" required>
            </div>

            <div class="form-group">
                <label for="previous_therapy">Have you talked to a mental health professional before?</label>
                <select id="previous_therapy" name="previous_therapy">
                    <option value="no">No, this is my first time seeking support</option>
                    <option value="yes-currently">Yes, I'm currently in therapy</option>
                    <option value="yes-past">Yes, I've been in therapy in the past</option>
                </select>
            </div>

            <button type="submit" class="submit-btn">Start Chatting 💬</button>
        </form>

        <div class="privacy-note">
            <strong>🔒 Your Privacy:</strong> This conversation is confidential and not stored permanently. However, this is an AI assistant and not a replacement for professional mental health care. If you're experiencing a mental health crisis, please contact a professional immediately.
        </div>
    </div>

    <script>
        // Mood selection
        const moodBtns = document.querySelectorAll('.mood-btn');
        const moodInput = document.getElementById('mood');

        moodBtns.forEach(btn => {
            btn.addEventListener('click', () => {
                moodBtns.forEach(b => b.classList.remove('selected'));
                btn.classList.add('selected');
                moodInput.value = btn.dataset.mood;
            });
        });

        // Form submission
        document.getElementById('intakeForm').addEventListener('submit', async (e) => {
            e.preventDefault();

            const formData = {
                name: document.getElementById('name').value,
                age: document.getElementById('age').value,
                concern: document.getElementById('concern').value,
                mood: document.getElementById('mood').value,
                previous_therapy: document.getElementById('previous_therapy').value
            };

            if (!formData.mood) {
                alert('Please select how you\'re feeling right now');
                return;
            }

            try {
                const response = await fetch('/submit_intake', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(formData)
                });

                const data = await response.json();

                if (data.status === 'success') {
                    window.location.href = '/chat';
                } else {
                    alert('Error: ' + data.message);
                }
            } catch (error) {
                alert('Connection error. Please try again.');
            }
        });
    </script>
</body>
</html>

📘 7️⃣ Chat Interface Overview (chat.html)

The chat interface includes:

✨ Header with user name  
✨ Mood-tracker button  
✨ End-session button  
✨ Sliding coping tools bar:
   – Breathing exercise  
   – Grounding  
   – Stress relief  
   – Interactive breathing animation  
   – Self-care prompts  

🗨️ Message display:
• User messages — right aligned  
• Bot messages — left aligned, formatted

💬 Typing Indicator  
Auto-resizing textarea  
Send on Enter  
Crisis alerts shown as rich messages  

🎭 Modals:
• Breathing exercise visual animation  
• Mood tracker emoji popup


📘 8️⃣ Formatting Enhancements (Frontend)

chat.html auto-formats bot messages:

• Converts **bold text**  
• Converts bullet points  
• Handles multi-line paragraphs  
• Renders structured responses cleanly  

This ensures mental-health guidance appears readable and professional.


In [ ]:
# ===============================
# CELL 3: Create chat.html with FIXED FORMATTING
# ===============================
%%writefile templates/chat.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🧠 Mental Health Support Chat</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Inter', 'Segoe UI', system-ui, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            height: 100vh;
            display: flex;
            flex-direction: column;
        }

        .chat-header {
            background: white;
            padding: 20px 30px;
            box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
            display: flex;
            justify-content: space-between;
            align-items: center;
            flex-wrap: wrap;
            gap: 15px;
        }

        .header-left {
            display: flex;
            align-items: center;
            gap: 15px;
        }

        .header-logo {
            font-size: 36px;
        }

        .header-info h2 {
            font-size: 20px;
            color: #2d3748;
        }

        .header-info p {
            font-size: 14px;
            color: #718096;
        }

        .header-right {
            display: flex;
            gap: 10px;
            align-items: center;
        }

        .mood-tracker-btn {
            padding: 10px 20px;
            background: #667eea;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-weight: 600;
            transition: all 0.3s ease;
        }

        .mood-tracker-btn:hover {
            background: #5568d3;
        }

        .end-session-btn {
            padding: 10px 20px;
            background: #fc8181;
            color: white;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            font-weight: 600;
            transition: all 0.3s ease;
        }

        .end-session-btn:hover {
            background: #f56565;
        }

        .chat-container {
            flex: 1;
            display: flex;
            flex-direction: column;
            max-width: 900px;
            width: 100%;
            margin: 20px auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 10px 40px rgba(0, 0, 0, 0.2);
            overflow: hidden;
        }

        .coping-tools-bar {
            background: #f7fafc;
            padding: 15px 20px;
            border-bottom: 1px solid #e2e8f0;
            display: flex;
            gap: 10px;
            overflow-x: auto;
            flex-wrap: wrap;
        }

        .tool-btn {
            padding: 8px 16px;
            background: white;
            border: 2px solid #667eea;
            color: #667eea;
            border-radius: 20px;
            cursor: pointer;
            font-size: 13px;
            font-weight: 600;
            transition: all 0.3s ease;
            white-space: nowrap;
        }

        .tool-btn:hover {
            background: #667eea;
            color: white;
            transform: translateY(-2px);
        }

        .messages-container {
            flex: 1;
            overflow-y: auto;
            padding: 30px;
            background: #f7fafc;
        }

        .message {
            margin-bottom: 20px;
            animation: slideIn 0.3s ease;
        }

        @keyframes slideIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }

        .message.user {
            text-align: right;
        }

        .message-content {
            display: inline-block;
            max-width: 70%;
            padding: 15px 20px;
            border-radius: 18px;
            font-size: 15px;
            line-height: 1.8;
        }

        .message.user .message-content {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border-bottom-right-radius: 4px;
        }

        .message.bot .message-content {
            background: white;
            color: #2d3748;
            border: 1px solid #e2e8f0;
            border-bottom-left-radius: 4px;
            text-align: left;
        }

        .message.bot .message-content strong {
            color: #667eea;
            font-weight: 600;
        }

        .input-container {
            padding: 20px;
            background: white;
            border-top: 1px solid #e2e8f0;
            display: flex;
            gap: 10px;
        }

        .message-input {
            flex: 1;
            padding: 15px;
            border: 2px solid #e2e8f0;
            border-radius: 12px;
            font-size: 15px;
            font-family: inherit;
            resize: none;
            max-height: 150px;
        }

        .message-input:focus {
            outline: none;
            border-color: #667eea;
        }

        .send-btn {
            padding: 15px 30px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 12px;
            cursor: pointer;
            font-weight: 600;
            font-size: 15px;
            transition: all 0.3s ease;
        }

        .send-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }

        .send-btn:disabled {
            opacity: 0.5;
            cursor: not-allowed;
        }

        .typing-indicator {
            display: none;
            padding: 15px 20px;
            background: white;
            border: 1px solid #e2e8f0;
            border-radius: 18px;
            border-bottom-left-radius: 4px;
        }

        .typing-indicator.active {
            display: inline-block;
        }

        .typing-dots {
            display: flex;
            gap: 5px;
        }

        .typing-dots span {
            width: 8px;
            height: 8px;
            background: #667eea;
            border-radius: 50%;
            animation: bounce 1.4s infinite ease-in-out;
        }

        .typing-dots span:nth-child(1) { animation-delay: -0.32s; }
        .typing-dots span:nth-child(2) { animation-delay: -0.16s; }

        @keyframes bounce {
            0%, 80%, 100% { transform: scale(0); }
            40% { transform: scale(1); }
        }

        .welcome-message {
            text-align: center;
            padding: 40px 20px;
            color: #718096;
        }

        .welcome-message h3 {
            color: #2d3748;
            margin-bottom: 10px;
        }

        /* Breathing Exercise Modal */
        .modal {
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0, 0, 0, 0.8);
            animation: fadeIn 0.3s ease;
        }

        .modal-content {
            background: white;
            margin: 5% auto;
            padding: 40px;
            border-radius: 20px;
            max-width: 500px;
            text-align: center;
            position: relative;
        }

        .close-modal {
            position: absolute;
            top: 15px;
            right: 20px;
            font-size: 28px;
            font-weight: bold;
            color: #718096;
            cursor: pointer;
            transition: color 0.3s;
        }

        .close-modal:hover {
            color: #2d3748;
        }

        .breathing-circle {
            width: 200px;
            height: 200px;
            border-radius: 50%;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            margin: 30px auto;
            display: flex;
            align-items: center;
            justify-content: center;
            color: white;
            font-size: 24px;
            font-weight: 600;
            transition: transform 4s ease-in-out;
        }

        .breathing-circle.inhale {
            transform: scale(1.5);
        }

        .breathing-circle.hold {
            transform: scale(1.5);
        }

        .breathing-circle.exhale {
            transform: scale(1);
        }

        .breathing-instructions {
            font-size: 18px;
            color: #2d3748;
            margin-top: 20px;
            font-weight: 600;
        }

        .breathing-counter {
            font-size: 48px;
            color: #667eea;
            font-weight: 700;
            margin: 10px 0;
        }

        .mood-modal .mood-options {
            display: flex;
            justify-content: center;
            gap: 15px;
            margin: 30px 0;
        }

        .mood-modal .mood-btn {
            font-size: 48px;
            cursor: pointer;
            transition: transform 0.3s;
            padding: 10px;
            border-radius: 10px;
        }

        .mood-modal .mood-btn:hover {
            transform: scale(1.2);
            background: #f7fafc;
        }

        .start-breathing-btn {
            padding: 15px 30px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 12px;
            cursor: pointer;
            font-weight: 600;
            font-size: 16px;
            margin-top: 20px;
        }

        .start-breathing-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }

        @media (max-width: 768px) {
            .chat-container { margin: 10px; border-radius: 15px; }
            .messages-container { padding: 20px; }
            .message-content { max-width: 85%; }
            .header-info h2 { font-size: 16px; }
            .coping-tools-bar { padding: 10px; }
            .tool-btn { font-size: 12px; padding: 6px 12px; }
        }
    </style>
</head>
<body>
    <div class="chat-header">
        <div class="header-left">
            <div class="header-logo">🧠</div>
            <div class="header-info">
                <h2>Mental Health Support</h2>
                <p>Chatting with {{ user_data.name }}</p>
            </div>
        </div>
        <div class="header-right">
            <button class="mood-tracker-btn" onclick="showMoodTracker()">Track Mood 😊</button>
            <button class="end-session-btn" onclick="endSession()">End Session</button>
        </div>
    </div>

    <div class="chat-container">
        <div class="coping-tools-bar">
            <button class="tool-btn" onclick="sendQuickMessage('breathing')">🌬️ Breathing Exercise</button>
            <button class="tool-btn" onclick="sendQuickMessage('grounding')">🧘 Grounding (5-4-3-2-1)</button>
            <button class="tool-btn" onclick="sendQuickMessage('stress relief')">⚡ Quick Stress Relief</button>
            <button class="tool-btn" onclick="startBreathingExercise()">💨 Interactive Breathing</button>
            <button class="tool-btn" onclick="sendQuickMessage('self-care tips')">💚 Self-Care Ideas</button>
        </div>

        <div class="messages-container" id="messagesContainer">
            <div class="welcome-message">
                <h3>Welcome, {{ user_data.name }}! 👋</h3>
                <p>I'm here to listen and support you. Feel free to share what's on your mind.<br>
                <strong>Try the coping tools above</strong> for quick stress relief!<br>
                Remember, I'm an AI assistant - for professional help, please contact a licensed therapist.</p>
            </div>
        </div>

        <div class="input-container">
            <textarea
                class="message-input"
                id="messageInput"
                placeholder="Type your message here... or use the coping tools above"
                rows="1"></textarea>
            <button class="send-btn" id="sendBtn" onclick="sendMessage()">Send ➤</button>
        </div>
    </div>

    <!-- Breathing Exercise Modal -->
    <div id="breathingModal" class="modal">
        <div class="modal-content">
            <span class="close-modal" onclick="closeBreathingExercise()">&times;</span>
            <h2>🌬️ Breathing Exercise</h2>
            <p>Follow the circle and breathe with me</p>

            <div class="breathing-circle" id="breathingCircle">Breathe</div>

            <div class="breathing-instructions" id="breathingInstructions">Click Start to Begin</div>
            <div class="breathing-counter" id="breathingCounter">4</div>

            <button class="start-breathing-btn" id="startBreathingBtn" onclick="runBreathingExercise()">
                Start Exercise
            </button>
        </div>
    </div>

    <!-- Mood Tracker Modal -->
    <div id="moodModal" class="modal mood-modal">
        <div class="modal-content">
            <span class="close-modal" onclick="closeMoodTracker()">&times;</span>
            <h2>😊 How are you feeling now?</h2>
            <p>Track your mood throughout our conversation</p>

            <div class="mood-options">
                <div class="mood-btn" onclick="trackMood('very-sad')" title="Very Sad">😢</div>
                <div class="mood-btn" onclick="trackMood('sad')" title="Sad">😔</div>
                <div class="mood-btn" onclick="trackMood('neutral')" title="Neutral">😐</div>
                <div class="mood-btn" onclick="trackMood('good')" title="Good">🙂</div>
                <div class="mood-btn" onclick="trackMood('great')" title="Great">😊</div>
            </div>

            <p style="margin-top: 20px; color: #718096; font-size: 14px;">
                Your mood has been tracked! This helps you see your emotional progress over time.
            </p>
        </div>
    </div>

    <script>
        const messagesContainer = document.getElementById('messagesContainer');
        const messageInput = document.getElementById('messageInput');
        const sendBtn = document.getElementById('sendBtn');

        // Auto-resize textarea
        messageInput.addEventListener('input', function() {
            this.style.height = 'auto';
            this.style.height = Math.min(this.scrollHeight, 150) + 'px';
        });

        // Send on Enter (Shift+Enter for new line)
        messageInput.addEventListener('keydown', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                sendMessage();
            }
        });

        function addMessage(content, isUser) {
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${isUser ? 'user' : 'bot'}`;

            if (isUser) {
                messageDiv.innerHTML = `<div class="message-content">${escapeHtml(content)}</div>`;
            } else {
                // THIS IS THE KEY FIX: Format bot messages properly
                let formattedContent = content;

                // Convert **text** to <strong>text</strong>
                formattedContent = formattedContent.replace(/\*\*(.*?)\*\*/g, '<strong>$1</strong>');

                // Convert bullet points to proper HTML with indentation
                formattedContent = formattedContent.replace(/^• (.+)$/gm, '<div style="margin-left: 20px; margin-bottom: 5px;">• $1</div>');

                // Convert double line breaks to paragraph spacing
                formattedContent = formattedContent.replace(/\n\n/g, '<br><br>');

                // Convert single line breaks to <br>
                formattedContent = formattedContent.replace(/\n/g, '<br>');

                messageDiv.innerHTML = `<div class="message-content">${formattedContent}</div>`;
            }

            messagesContainer.appendChild(messageDiv);
            messagesContainer.scrollTop = messagesContainer.scrollHeight;
        }

        function showTypingIndicator() {
            const typingDiv = document.createElement('div');
            typingDiv.className = 'message bot';
            typingDiv.id = 'typingIndicator';
            typingDiv.innerHTML = `
                <div class="typing-indicator active">
                    <div class="typing-dots">
                        <span></span>
                        <span></span>
                        <span></span>
                    </div>
                </div>
            `;
            messagesContainer.appendChild(typingDiv);
            messagesContainer.scrollTop = messagesContainer.scrollHeight;
        }

        function removeTypingIndicator() {
            const typingIndicator = document.getElementById('typingIndicator');
            if (typingIndicator) {
                typingIndicator.remove();
            }
        }

        async function sendMessage() {
            const message = messageInput.value.trim();

            if (!message) return;

            // Add user message
            addMessage(message, true);
            messageInput.value = '';
            messageInput.style.height = 'auto';

            // Disable input
            sendBtn.disabled = true;
            messageInput.disabled = true;

            // Show typing indicator
            showTypingIndicator();

            try {
                const response = await fetch('/send_message', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ message: message })
                });

                const data = await response.json();

                // Remove typing indicator
                removeTypingIndicator();

                if (data.status === 'success') {
                    addMessage(data.response, false);
                } else {
                    addMessage('I apologize, but I encountered an error. Please try again.', false);
                }
            } catch (error) {
                removeTypingIndicator();
                addMessage('Connection error. Please check your internet and try again.', false);
            } finally {
                sendBtn.disabled = false;
                messageInput.disabled = false;
                messageInput.focus();
            }
        }

        function sendQuickMessage(message) {
            messageInput.value = message;
            sendMessage();
        }

        function escapeHtml(text) {
            const div = document.createElement('div');
            div.textContent = text;
            return div.innerHTML;
        }

        // Breathing Exercise Functions
        function startBreathingExercise() {
            document.getElementById('breathingModal').style.display = 'block';
        }

        function closeBreathingExercise() {
            document.getElementById('breathingModal').style.display = 'none';
            stopBreathing();
        }

        let breathingInterval;
        let breathingActive = false;

        function runBreathingExercise() {
            if (breathingActive) {
                stopBreathing();
                return;
            }

            breathingActive = true;
            const btn = document.getElementById('startBreathingBtn');
            btn.textContent = 'Stop Exercise';

            const circle = document.getElementById('breathingCircle');
            const instructions = document.getElementById('breathingInstructions');
            const counter = document.getElementById('breathingCounter');

            let phase = 0;
            let count = 4;

            const phases = [
                { name: 'Breathe In', duration: 4, class: 'inhale' },
                { name: 'Hold', duration: 7, class: 'hold' },
                { name: 'Breathe Out', duration: 8, class: 'exhale' }
            ];

            function breathe() {
                const currentPhase = phases[phase];
                instructions.textContent = currentPhase.name;
                counter.textContent = count;

                circle.className = 'breathing-circle ' + currentPhase.class;

                count--;

                if (count < 0) {
                    phase = (phase + 1) % 3;
                    count = phases[phase].duration;
                }
            }

            breathe();
            breathingInterval = setInterval(breathe, 1000);
        }

        function stopBreathing() {
            breathingActive = false;
            clearInterval(breathingInterval);
            document.getElementById('startBreathingBtn').textContent = 'Start Exercise';
            document.getElementById('breathingInstructions').textContent = 'Click Start to Begin';
            document.getElementById('breathingCounter').textContent = '4';
            document.getElementById('breathingCircle').className = 'breathing-circle';
        }

        // Mood Tracker Functions
        function showMoodTracker() {
            document.getElementById('moodModal').style.display = 'block';
        }

        function closeMoodTracker() {
            document.getElementById('moodModal').style.display = 'none';
        }

        async function trackMood(mood) {
            try {
                await fetch('/update_mood', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ mood: mood })
                });

                const moodEmojis = {
                    'very-sad': '😢',
                    'sad': '😔',
                    'neutral': '😐',
                    'good': '🙂',
                    'great': '😊'
                };

                addMessage(`Mood tracked: ${moodEmojis[mood]} Thank you for sharing how you're feeling!`, false);
                closeMoodTracker();
            } catch (error) {
                console.error('Mood tracking error:', error);
            }
        }

        // Close modals on outside click
        window.onclick = function(event) {
            if (event.target.classList.contains('modal')) {
                event.target.style.display = 'none';
                stopBreathing();
            }
        }

        async function endSession() {
            if (confirm('Are you sure you want to end this session? Your conversation will not be saved.')) {
                try {
                    await fetch('/end_session', { method: 'POST' });
                    window.location.href = '/';
                } catch (error) {
                    window.location.href = '/';
                }
            }
        }

        // Focus on input on load
        messageInput.focus();
    </script>
</body>
</html>

📘 9️⃣ Killing Running Flask/ngrok Sessions

Before starting a new run:

!pkill -f flask || echo "No flask running"  
!pkill -f ngrok || echo "No ngrok running"

Why?
• Prevents "Address already in use :8000” errors  
• Ensures ngrok creates a fresh tunnel  


In [ ]:
# ===============================
# 6️⃣ Kill any previous processes
# ===============================
!pkill -f flask || echo "No flask running"
!pkill -f ngrok || echo "No ngrok running"


📘 🔟 Check Port Usage

Use this to find which PID is using port 8000:

!lsof -i :8000

Example:
python3  376  ...  TCP *:8000 (LISTEN)


In [ ]:
!lsof -i :8000

📘 1️⃣1️⃣ Free Port 8000

Kill the process to free the port:

!kill -9 376


In [ ]:
!kill -9 376

📘 1️⃣2️⃣ Run Flask Server in Background

Start the backend without blocking your notebook:

!nohup python app.py > flask.log 2>&1 &


In [ ]:
# ===============================
# 7️⃣ Run Flask in the background
# ===============================
!nohup python app.py > flask.log 2>&1 &

📘 1️⃣3️⃣ Ngrok Setup (Token Removed)

Users must add their OWN ngrok token.

Steps:

1. Sign up:
   https://dashboard.ngrok.com/signup

2. Get your auth token:
   https://dashboard.ngrok.com/get-started/your-authtoken

3. Paste into code:

conf.get_default().auth_token = "YOUR_NGROK_TOKEN"

4. Start the tunnel:

public_url = ngrok.connect(8000)

This generates a public chat link:

https://abcd-1234.ngrok-free.app


In [ ]:
# ===============================
# 8️⃣ Start ngrok tunnel
# ===============================
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN"  # 🔑 replace with your token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check logs (optional)
# ===============================
!sleep 3 && tail -n 20 flask.log


📘 1️⃣4️⃣ Log Debugging

Check last 20 log lines:
!tail -n 20 flask.log

Check last 50 log lines:
!tail -n 50 flask.log

Useful for:
• Model load issues  
• JSON errors  
• Frontend rendering issues  
• Session errors  


In [ ]:
!tail -n 50 flask.log

📘 1️⃣5️⃣ Project Complete 🎉

Your AI-Powered Mental Health Support Chatbot is ready.

Users can:
✔ Chat with empathetic AI  
✔ Receive personalized mental-health guidance  
✔ Access coping strategies  
✔ Track emotional progress  
✔ View crisis resources when needed  

⚠️ Reminder:
This tool is ONLY for emotional support.
Not a substitute for licensed professionals.


In [ ]:
📘 1️⃣6️⃣ Helpful Resources

DialoGPT:
https://huggingface.co/microsoft/DialoGPT-medium

BlenderBot:
https://huggingface.co/facebook/blenderbot-400M-distill

Crisis Help:
https://findahelpline.com

Flask Docs:
https://flask.palletsprojects.com/

Ngrok Docs:
https://pyngrok.readthedocs.io/en/latest/
